In [6]:
import stockutils
import pandas_datareader, re, sqlite3

In [7]:
class StockData:
    def __init__(self, se, sql = False, update = False):
        self.company_table = stockutils.company_table(se = se, update = update)
        self.se = se
        self.sql = sql
   
    def lookup_ticker(self, company_name):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        if isinstance(company_name,str):
            company_name = [company_name]
        result_table = [] 
        if not self.sql:
            for name in company_name:
                name = r'.*\s*'+name.lower()+r'\s.*'
                for row in self.company_table:
                    match = re.search(name,row[1].lower()) 
                    if match:
                        result_table.append((row[0],row[1]))
        else:
            conn = sqlite3.connect('data/companysql' + self.se + '.sql')
            c = conn.cursor()
            for name in company_name:
                query = ('%'+name+' %',)
                for row in c.execute('SELECT symbol,name FROM companytable WHERE lower(name) LIKE ?', query):
                    result_table.append(row)
            conn.close()
        return result_table.pop() if len(result_table) == 1 else result_table

In [12]:
x = StockData('nasdaq', sql = True)

In [4]:
y = StockData('amex', sql = True)

In [15]:
z = StockData('nyse')

In [13]:
x.lookup_ticker('Nvidia')

('NVDA', 'NVIDIA Corporation')

In [14]:
x.lookup_ticker(['Intel', 'Nvidia', 'Apple', 'Microsoft'])

[('INTC', 'Intel Corporation'),
 ('NVDA', 'NVIDIA Corporation'),
 ('AAPL', 'Apple Inc.'),
 ('MSFT', 'Microsoft Corporation')]

In [16]:
z.lookup_ticker("Sporting")

('DKS', 'Dick&#39;s Sporting Goods Inc')